# Language Models
## Homework 1: Language Modeling

**Instructor**: Pavlos Protopapas<br />
**Maximum Score**: 70

<hr style="height:2pt">


## INSTRUCTIONS

- This homework is a notebook. Download and work on it on your local machine or work on it in Colab.

- This homework should be submitted in pairs.

- Ensure you and your partner together have submitted the homework only once. Multiple submissions of the same work will be penalised and will cost you 2 points.

- Please restart the kernel and run the entire notebook again before you submit.

- Running cells out of order is a common pitfall in Jupyter Notebooks. To make sure your code works restart the kernel and run the whole notebook again before you submit. 

- To submit the homework, either one of you upload the working notebook on edStem and click the submit button on the bottom right corner.

- Submit the homework well before the given deadline. Submissions after the deadline will not be graded.

- We have tried to include all the libraries you may need to do the assignment in the imports statement at the top of this notebook. We strongly suggest that you use those and not others as we may not be familiar with them.

- Comment your code well. This would help the graders in case there is any issue with the notebook while running. It is important to remember that the graders will not troubleshoot your code. 

- Please use .head() when viewing data. Do not submit a notebook that is **excessively long**. 

- In questions that require code to answer, such as "calculate the $R^2$", do not just output the value from a cell. Write a `print()` function that includes a reference to the calculated value, **not hardcoded**. For example: 
```
print(f'The R^2 is {R:.4f}')
```
- Your plots should include clear labels for the $x$ and $y$ axes as well as a descriptive title ("MSE plot" is not a descriptive title; "95 % confidence interval of coefficients of polynomial degree 5" is).

- **Ensure you make appropriate plots for all the questions it is applicable to, regardless of it being explicitly asked for.**

<hr style="height:2pt">

## **Names of the people who worked on this homework together**
#### / Names here/

## **HOMEWORK QUIZ**

**For each part of the homework, there is an associated quiz on edStem. You are required to attempt that after completing each section of this homework.**

## **Setup Notebook**

**Imports**

In [ ]:
import requests
import re
import os
import zipfile
import collections
import numpy as np
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline
from IPython.core.display import HTML

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.keras.models import Model
from collections import defaultdict

from sklearn.model_selection import train_test_split

**Verify Setup**

In [ ]:
# Enable/Disable Eager Execution
# Reference: https://www.tensorflow.org/guide/eager
# TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, 
# without building graphs

#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.enable_eager_execution()

print("tensorflow version", tf.__version__)
print("keras version", tf.keras.__version__)
print("Eager Execution Enabled:", tf.executing_eagerly())

# Get the number of replicas 
strategy = tf.distribute.MirroredStrategy()
print("Number of replicas:", strategy.num_replicas_in_sync)

devices = tf.config.experimental.get_visible_devices()
print("Devices:", devices)
print(tf.config.experimental.list_logical_devices('GPU'))

print("GPU Available: ", tf.config.list_physical_devices('GPU'))
print("All Physical Devices", tf.config.list_physical_devices())

# Better performance with the tf.data API
# Reference: https://www.tensorflow.org/guide/data_performance
AUTOTUNE = tf.data.experimental.AUTOTUNE


    
## **PART 1 [35 points]: Language Modeling using ngrams**
<br />    

You have been tasked with developing a language model to complete messages that for some reason arrive incomplete to a disaster response station. Given the delicate situation, you will have to be extra careful. Each word in the sentence conveys a lot of information, and improper handling of the data could cause someone to come to harm. 

Your language model will be based on bigrams. You'll develop your own sub-word tokenization to analyze disaster messages from multiple natural disasters. All the sentences are translated into English.
    
    

    

### **PART 1: Questions**
<br />

### **1.1 [5 points] PREPROCESS THE DATASET**
<br />

**1.1.1** - Read in the dataset `disaster_response_messages_training.csv` and select only the column "message" and display the head of the DataFrame.
<br /><br />

**1.1.2** - Define a function `clean_data` that takes the data frame as input, converts the characters to lower case and removes any non-alphanumeric characters,  adds the start token `<s>` and the end token `</s>` to every sentence (row) in the data frame and returns the processed data frame.
<br />
**Sample Input:** "Is there a dog^ on the road?" <br />
**Sample Output:** "\<s> is there a dog on the road \</s>"
<br /><br />


**1.1.3** - Split the dataset into train and test sets. The proportion should be 0.95 and 0.05, respectively. You will create the language model based on the train set and validate your results on the test set.
<br /><br />    



    
### **1.2 [8 points] TOKENIZE AND COUNT**
<br />

In this section, you will create three different tokenizers that you will build LM based on. The tokenization functions must divide the text into tokens, count their frequency and return a dictionary with a mapping of token to number.
<br />

**1.2.1** - Create your own tokenization function ('tokenizer_1') based on whitespace. Pick the top 1000 tokens with the highest frequency to include in the vocabulary. Include the `<UNK>` token for out of the vocabulary (OOV) words.  <br /><br />

**1.2.2** - Create a second tokenization function ('tokenizer_2') based on whitespace, but do not limit the vocabulary size.
<br /><br />

**1.2.3** - Create a third tokenization function ('tokenizer_3') based on sub-words. You have to define a set of common sub-words in the English language, for example, the subtokens _ing_ and _n't_. Here again, do not limit the vocabulary size.

In this example, the sentence "_It is raining outside_" would be tokenized as [_It_, _is_, _rain_, _ing_, _outside_ ].


**Note:** Use words only from the train data to create the vocabulary. Add an additional `<UNK>` token for **1.2.2** and **1.2.3** to handle new words found in the test set. Remove the empty character token as it provides no information
<br /><br />
    
### **1.3 [6 points] CONSTRUCTING BIGRAMS**
<br />

**1.3.1** - Using each of the tokenizer functions you created, split each sentence into tokens in their numerical representation. 
<br /><br />

**1.3.2** - For each tokenizer, count the occurances of each bigram (w1,w2) in the train dataset and divide them by the total occurences of the first word (w1). This will give you the probability of each bigram.
<br /><br />
    
    
### **1.4 [8 points] PREDICTING THE NEXT WORDS**
<br />

**1.4.1** - Simulate incomplete messages by dividing each sentence of the test set in two. The first $75\%$ will represent the received message, and the final $25\%$ will convey the missing information. You will use this dataset to evaluate the predictions of your language model.
    
For example in the sentence: "*I will go out on a vacation, now that my semester ended.*"

The first 75% will be "*I will go out on a vacation, now*"

The last 25% will be "*that my semeter ended*"

Your aim is to predict the last part by giving your model the first "part" of the sentence.


**Note:** In an n-gram language model, only the last $n-1$ words are used to make a prediction. 

For example, for the above sentence, if you are using bigrams, the input to your model would only be "now" and you are expected to predict "that". 
<br /><br />

    
**1.4.2** - Given 5 sentences from the previous question (test set), predict the next word. 
Append this predicted word to the input sequence and predict the next one. Repeat this process until you reach the 10th predicted token or the end of a sentence (end of a sentence - `</s>`). Compare your results qualitatively with the original sentences. Do the results make sense wrt the context and semantics?

Repeat this for all the models built using different tokenization techniques.


**Note:** For model 2 and 3 (using tokenizer 2 and 3), if there is an `<UNK>` word as the last word in the test set, predict the next word based on unigram probabilities (calculate this by using word count from Section **1.2**).
<br /><br />

**1.4.3** - Repeat the same exercise, for all 3 models, but this time, the next token will be sampled from a distribution given by the bigram frequency. Compare and comment on the results?


**Hint:** In a model of two bigrams with frequencies 0.7 and 0.3, a deterministic prediction will only predict the first bigram. Sampling from a distribution, will enable the model to predict the second bigram with a probability of 0.3. In this way we can still predict infrequent tokens. 
<br /><br />
    

### **1.5 [5 points] EVALUATE THE LANGUAGE MODELS**
<br />

    
**1.5.1** - For each of your models, compute the average perplexity on the test set (These are the complete test messages as tokenized in 1.3.1, **not** the incomplete test messages from 1.4.1). If the tokens of the test set are not present in the train split, define a minimum probability (smoothing). Based on this metric, which model is better?

**Note:** Use the bigram probabilities for this. N (from the perplexity formula) - Number of words in the sentence.
<br /><br />

**1.5.2** - Given the perplexities, which model do you think is better? Why do you think so? Does this reflect the quality of the prediction as seen in part 1.4? 
 What is the effect of UNK words?

<br /><br />

### **1.6 [3 points] HOMEWORK QUIZ**
<br />

After attempting this part of the homework, answer the questions on edStem. All the questions depend on this part of the homework and you will not be able to answer them without attempting this part.

<br /><br />

</div>

---

## **PART 1: Solutions**

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
### **1.1 [5 points] PREPROCESS THE DATASET**
<br />

**1.1.1** - Read in the dataset `disaster_response_messages_training.csv`. Select only the column "message" and display the head of the DataFrame.
<br />

If you want to download the file you can get it from [here](https://drive.google.com/uc?id=1JRWRywWDZRdxTG1n9H9xTHXNapNdrTOJ&export=download)
</div>

In [ ]:
file_path = "https://drive.google.com/uc?id=1JRWRywWDZRdxTG1n9H9xTHXNapNdrTOJ&export=download"
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
**1.1.2** - Define a function `clean_data` that takes the data frame as input, converts the characters to lower case and removes any non-alphanumeric,  adds the start token `<s>` and the end token `</s>` to every sentence (row) in the data frame and returns the processed data frame.
<br />
**Sample Input:** "Is there a dog^ on the road?" <br />
**Sample Output:** "\<s> is there a dog on the road \</s>"
    
</div>

In [ ]:
# Your code here


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.1.3** - Split the dataset into train and test sets. The proportion should be 0.95 and 0.05, respectively. You will create the language model based on the train set and validate your results on the test set.
<br /><br />  

</div>

In [ ]:
# Your code here


<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **1.2 [8 points] TOKENIZE AND COUNT**
<br />

In this section, you will create three different tokenizers and build an LM based on each one of them. The tokenization functions must divide the text into tokens, count their frequency and return a dictionary with a mapping of token to number.
<br /><br />
    
**Note:** Use words only from the train data to create the vocabulary. Add an additional `<UNK>` token for **1.2.2** and **1.2.3** to handle new words found in the test set. Remove the empty character token as it provides no information. 
<br /><br />

**1.2.1** - Create your own tokenization function ('tokenizer_1') based on whitespace. Pick the top 1000 tokens with the highest frequency to include in the vocabulary. Include the `<UNK>` token for out of the vocabulary (OOV) words. 
<br /><br />
    
</div> 

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.2.2** - Create a second tokenization function ('tokenizer_2') based on whitespace, but do not limit the vocabulary size.
<br /><br />
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.2.3** - Create a third tokenization function ('tokenizer_3') based on sub-words. You have to define a set of common sub-words in the English language, for example, the subtokens _ing_ and _n't_.  Here again, do not limit the vocabulary size.
    
In this example, the sentence "_It is raining outside_" would be tokenized as [_It_, _is_, _rain_, _ing_, _outside_ ].
<br /><br />
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **1.3 [7 points] CONSTRUCTING BIGRAMS**
<br />

**1.3.1** - Using each of the tokenizer functions you created, split each sentence into tokens in their numerical representation. 
<br /><br />
    
</div>

**Tokenizer_1**

In [ ]:
# Your code here

**Tokenizer_2**

In [ ]:
# Your code here

**Tokenizer_3**

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.3.2** - For each tokenizer, count the occurances of each bigram (w1,w2) in the train dataset and divide them by the total occurences of the first word (w1). This will give you the probability of each bigram.
<br /><br />
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **1.4 [9 points] PREDICTING THE NEXT WORDS**
<br />

**1.4.1** - Simulate incomplete messages by dividing each sentence of the test set in two. The first $75\%$ will represent the received message, and the final $25\%$ will convey the missing information. You will use this dataset to evaluate the predictions of your language model.
    
For example in the sentence: "*I will go out on a vacation, now that my semester ended.*"

The first 75% will be "*I will go out on a vacation, now*"

The last 25% will be "*that my semeter ended*"

Your aim is to predict the last part by giving your model the first "part" of the sentence.


**Note:** In an n-gram language model, only the last $n-1$ words are used to make a prediction. 

For example, for the above sentence, if you are using bigrams, the input to your model would only be "now" and you are expected to predict "that". 
<br /><br />
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">


**1.4.2** - For 5 sentences from the previous question (test set), predict the next word. 
Append this predicted word to the input sequence and predict the next one. Repeat this process until you reach the 10th predicted token or the end of a sentence (end of a sentence - `</s>`). Compare your results qualitatively with the original sentences. Do the results make sense wrt the context and semantics?

Repeat this for all the models built using different tokenization techniques.


**Note:** For model 2 and 3  (using tokenizer 2 and 3),, if there is an `<UNK>` word as the last word in the test set, predict the next word based on unigram probabilities (calculate this by using word count from Section **1.2**)
<br /><br />
    
</div>

In [ ]:
# Your code here

**Tokenizer1**

In [ ]:
# Your code here

**Tokenizer2**

In [ ]:
# Your code here

**Tokenizer3**

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.4.3** - Repeat the same exercise, but this time, the next token will be _sampled_ from a distribution given by the bigram frequency. Are the results better?

In a model of two bigrams with frequencies 0.7 and 0.3, a deterministic prediction will only predict the first bigram. Sampling from a distribution, will enable the model to predict the second bigram with a probability of 0.3. In this way we can still predict infrequent tokens. 
<br /><br />
    
</div>

In [ ]:
# Your code here

**Tokenizer_1**

In [ ]:
# Your code here

**Tokenizer_2**

In [ ]:
# Your code here

**Tokenizer_3**

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **1.5 [6 points] EVALUATE THE LANGUAGE MODELS**
<br />

    
**1.5.1** - For each of your models, compute the average perplexity on the test set (These are the complete test messages as tokenized in 1.3.1, **not** the incomplete test messages from 1.4.1). If the tokens of the test set are not present in the train split, define a minimum probability (smoothing). Based on this metric, which model is better?

**Note:** Use the bigram probabilities for this. N - (from the perplexity formula) Number of words in the sentence.
<br /><br />
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**1.5.2** - Given the perplexities, which model do you think is better? Why do you think so? Does this reflect the quality of the prediction as seen in part 1.4? 
 What is the effect of UNK words?
<br /><br />
    
</div>

Type your answer here

___
___


## **PART 2 [35 points] : Language Modelling using RNNs**
<br />    

In this part of the homework, you are to build and train a new language model. For this, we will be using Prof. Protopapas's famous texts which end with `...` for prediction. Here, you will preprocess a data corpus and train your simple RNN network with it. With this network you will try to predict what he meant when he typed `...`. For this task we will use a form of transfer learning: first training a network on the larger dataset, such as IMDB reviews, and then 'fine tuning' the network to the professor's texts.
<br /><br />
    
</div>

## **PART 2: Questions**
<br />

### **2.1 [2 points] PREPROCESS THE DATASET**
<br />

**2.1.1** - Read in the dataset `imdb.csv`. Create a new dataframe by splitting each review into individual sentences. The sentences can be delimited by different characters such as period and question mark (eroteme). Call this column as `text` in the new dataframe.
<br /><br />

**2.1.2** - Define a function `clean_data` that takes the new dataframe as input and removes all html tags and non-alphanumeric characters from the dataframe. Additionally, convert all characters to lower case. Remove all the sentences where the number of words is less than 10 and higher than 30. Finally, add the start token `<s>` and the end token `</s>` to every sentence (row) in the dataframe. Return the processed the dataframe. 
<br /><br />

### **2.2 [2 points] TOKENIZE THE DATASET**
<br />

**2.2.1** - Instantiate a Tokenizer for the dataset using `tensorflow.keras.preprocessing.text.Tokenizer` with a vocabulary size of 5000.  Do **not** use an additional token for unknown, out of vocabulary words (oov). That is, you will **not** have the equivalent of the `<UNK>` token.

*Hint:* Remove all filters from the function as we have already perfomed data cleaning. Set the value of filter to be `''`.
<br /><br />

**2.2.2** - Fit the tokenizer on the dataset and get the sequence representation of each sentence.
<br /><br />

### **2.3 [10 points] MODELLING THE DATA**
<br />

**2.3.1** - The first step is to split the dataset into the predictors ($X$) and the response ($Y$). The predictors for each observation (sentence) are all tokens in that sentence _except_ the **last** token. The response for a given sentence is all tokens in that sentence _except_ the **first**. Using `tf.keras.preprocessing.sequence.pad_sequences` post-pad each sequence in $X$ and $Y$ to a length of 30.
    
**Hint:** You may need to use `tf.convert_to_tensor` on the objects returned by the padding operation. 
```
Example:
if token for <s> = 1 and </s> = 2
sentence_i = [1, 48, 2498, 22, 16, 4, 4, 1554, 149, 14, 22, 2]
x_i = [1,  48,   2498, 22, 16, 4, 4,    1554, 149, 14, 22, 0, ..., 0]
y_i = [48, 2498, 22,   16, 4,  4, 1554, 149,  14,  22, 2,  0, ..., 0]
```
<br /><br />

**2.3.2** - Define a simple RNN model that has an embedding layer with an embedding dimension of 300. You can define any number of RNN layers. The output of the RNN model will be a dense layer with size of the vocabulary and softmax activation. Using the functional API here may make it easier to reuse parts of the network later on.
<br /><br />

**2.3.3** - Train the model with the $X$ and $Y$ data formed in 2.3.1. Use a validation split of 0.2. The choice of number epochs and batch size is left to you.
<br /><br />

**2.3.4** - Plot the train and validation loss from the training history.
<br /><br />

### **2.4 [9 points] PREDICTING THE NEXT WORD**
<br />

**2.4.1** -Read the dataset `pp_text.csv`. Add only the  start token to each line, remove the last word and tokenize it using the tokenizer fit previously. Convert each sentence to a sequence vector and post-pad to a length of 30. This will be the input for the prediction phase.
<br /><br />

**2.4.2** - For predicting the next word, use the trained RNN model from above. 

NOTE - Based on your implementation, the output of the RNN model might have to be different from that of your trained network. You can make use of Keras function API for this.
<br /><br />

**2.4.3** - Choose any sentence from the list of Pavlos' texts to predict the next word. Input this to the RNN model built for prediction and print the predicted word. Try this out with multiple sentences.
<br /><br />

**2.4.4** - Do you notice any pattern in the predicted words? Do they seem approriate to the context of the texts as you understand it? What do you attribute this discrepency to? How can you resolve it?

Answer in less than 150 words.
<br /><br />

### **2.5 [6 points] TRAINING AND PREDICTING WITH A DIFFERENT DATASET**
<br />

**2.5.1** - Read the dataset `cleaned_sarcasm.csv`. This dataset has been preprocessed for you, all you need to do is tokenize, convert to sequence and pad it, similar to 2.2.1, 2.2.2 and 2.3.1.
<br /><br />

**2.5.2** - Train your RNN model with this data and plot the train and validation trace plot. This part is similar to 2.3.2, 2.3.3 and 2.3.4.
<br /><br />

**2.5.3** - Repeat 2.4.1, 2.4.2 and 2.4.3 with the RNN model trained using the new dataset.
<br /><br />

**2.5.4** - How do the results with the new dataset compare to the previous ones? Why do you think so? 

Answer in less than 100 words.
<br /><br />
    
### **2.6 [3 points] COMPLETING THE SENTENCE**
<br />

**2.6.1** Until now we have predicted a single word for a given sentence. However, what if he meant more than one word when he typed in `...`

We will now predict multiple words for each input sentence. To do this we will first predict one word, append this word to the input text and then predict one more with the updated input. Continue doing this for 5 words or until the end token `</s>` (whichever comes first). 
<br /><br />

### **2.7 [3 points] HOMEWORK QUIZ**
<br />
After attempting this part of the homework, answer the questions on edStem. All the questions depend on this part of the homework and you will not be able to answer them without attempting this part.

<br />



---

## **PART 2: Solutions**

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
### **2.1 [2 points] PREPROCESS THE DATASET**
<br />    

**2.1.1** - Read in the dataset `imdb.csv`. Create a new dataframe by splitting each review into individual sentences. The sentences can be delimited by different characters such as period and question mark (eroteme). Call this column as `text` in the new dataframe.
</div>

In [ ]:
# Read the data
file_path = "https://drive.google.com/uc?id=1QDSIaV4iERVgc3b0xkW0u7EuTyQ8vncm&export=download"
data = pd.read_csv(file_path, encoding='latin1')
data.head()

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.1.2** - Define a function `clean_data` that takes the new dataframe as input and removes all html tags and non-alphabetic characters from the dataframe. Additionally, convert all characters to lower case. Remove all the sentences where the number of words is less than 10 and higher than 30. Finally, add the start token `<s>` and the end token `</s>` to every sentence (row) in the dataframe. Return the processed the dataframe. 
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
### **2.2 [2 points] TOKENIZE THE DATASET**
    
<br />

**2.2.1** - Instantiate a Tokenizer for the dataset using `tensorflow.keras.preprocessing.text.Tokenizer` with a vocabulary size of 5000.Do **not** use an additional token for unknown, out of vocabulary words (oov). That is, you will **not** have the equivalent of the `<UNK>` token.

*Hint:* Remove all filters from the function as we have already perfomed data cleaning. Set the value of filter to be `''`.
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.2.2** - Fit the tokenizer on the dataset and get the sequence representation of each sentence.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
### **2.3 [10 points] MODELLING THE DATA**
    
**2.3.1** - The first step is to split the dataset into the predictors ($X$) and the response ($Y$). The predictors for each observation (sentence) are all tokens in that sentence _except_ the **last** token. The response for a given sentence is all tokens in that sentence _except_ the **first**. Using `tf.keras.preprocessing.sequence.pad_sequences` post-pad each sequence in $X$ and $Y$ to a length of 30.
    
```
Example:
if token for <s> = 1 and </s> = 2
sentence_i = [1, 48, 2498, 22, 16, 4, 4, 1554, 149, 14, 22, 2]
x_i = [1,  48,   2498, 22, 16, 4, 4,    1554, 149, 14, 22, 0, ..., 0]
y_i = [48, 2498, 22,   16, 4,  4, 1554, 149,  14,  22, 2,  0, ..., 0]
```
**Hint:** You may need to use `tf.convert_to_tensor` on the objects returned by the padding operation. 
</div>    

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">
    
**2.3.2** - Define a simple RNN model that has an embedding layer with an embedding dimension of 300. You can define any number of RNN layers. The output of the RNN model will be a dense layer with size of the vocabulary and softmax activation. Using the functional API here may make it easier to reuse parts of the network later on.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.3.3** - Train the model with the $X$ and $Y$ data formed in 2.3.1. Use a validation split of 0.2. The choice of number epochs and batch size is left to you.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.3.4** - Plot the train and validation loss from the training history.

</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

### **2.4 [10 points] PREDICTING THE NEXT WORD**
    
<br />
    
**2.4.1** - Read the dataset `pp_text.csv`. Add only the  start token to each line, remove the last word and tokenize it using the tokenizer fit previously. Convert each sentence to a sequence vector and post-pad to a length of 30. This will be the input for the prediction phase.
    
</div>

In [ ]:
# Read the data
file_path = "https://drive.google.com/uc?id=1sFolPhc31mqvCxC3JxORmur21e28XeA-&export=download"
df_pred = pd.read_csv(file_path)
df_pred.head()

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">


**2.4.2** - Define a simple RNN model with the trained weights of the trained RNN model for predicting the next word. You can make use of Keras function API to reuse the previously written code. The output of this new model is the last element of the RNN output defined earlier.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.4.3** - Choose any sentence from the list of Prof. Protopapas's texts to predict the next word. Input this to the RNN model built for prediction and print the predicted word. Try this out with multiple sentences.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.4.4** - Do you notice any pattern in the predicted words? Do they seem approriate to the context of the texts as you understand it? What do you attribute this discrepency to? How can you resolve it?

Answer in less than 150 words.
    
</div>

**Type your answer here**



<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">


### **2.5 [7 points] TRAINING AND PREDICTING WITH A DIFFERENT DATASET**
<br />
    
**2.5.1** - Read the dataset `cleaned_sarcasm.csv`. This dataset has been preprocessed for you, all you need to do is tokenize, convert to sequence and pad it, similar to 2.2.1, 2.2.2 and 2.3.1.
    
</div>

In [ ]:
# Read the data
file_path = "https://drive.google.com/uc?id=1kHZIXEcf_t0t2GcxtafF3mL0kuoC4etf&export=download"
df = pd.read_csv(file_path)
df.head()

In [ ]:
# Your code here

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.5.2** - Train your RNN model with this data and plot the train and validation trace plot. This part is similar to 2.3.2, 2.3.3 and 2.3.4.
    
</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.5.3** - Repeat 2.4.1, 2.4.2 and 2.4.3 with the RNN model trained using the new dataset.

</div>

In [ ]:
# Your code here

<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">

**2.5.4** - How do the results with the new dataset compare to the previous ones? Why do you think so? 

Answer in less than 100 words.
    
</div>

**Type your answer here**



<div class="alert alert-block alert-danger" style="color:black;background-color:#E7F4FA">


### **2.6 [4 points] COMPLETING THE SENTENCE**
<br />

**2.6.1** Until now we have predicted a single word for a given sentence. However, what if he meant more than one word when he typed in `...`

We will now predict multiple words for each input sentence. To do this we will first predict one word, append this word to the input text and then predict once more with the updated input. Continue doing this to predict 5 words or until the end token `</s>` (whichever comes first). 

</div>

In [ ]:
# Your code here